In [5]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications import VGG16
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.optimizers import Adam

path = 'faces-to-aug'


base_model= VGG16(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dropout(0.5)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dropout(0.5)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x=Dropout(0.5)(x)
x=Dense(256,activation='relu')(x) #dense layer 4
x=Dropout(0.5)(x)
x=Dense(128,activation='relu')(x) #dense layer 5
preds=Dense(8,activation='softmax')(x) #final layer with softmax activation
model=Model(inputs=base_model.input,outputs=preds)
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    
    
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        path,
        target_size=(223, 223),
        batch_size=32,
        class_mode='categorical',
        save_to_dir='keras-faces',
        save_prefix='aug',
        save_format='jpeg',
        shuffle=True)

# validation_generator = test_datagen.flow_from_directory(
#         'data/validation',
#         target_size=(223, 223),
#         batch_size=32,
#         class_mode='binary')
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n
model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs=50)
#         validation_data=validation_generator,
#         validation_steps=800)

Found 288 images belonging to 8 classes.
Epoch 1/50


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node block1_conv1_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1_1/convolution-0-TransposeNHWCToNCHW-LayoutOptimizer, block1_conv1_1/kernel/read)]]
	 [[{{node metrics_1/acc/Mean/_557}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_1162_metrics_1/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]